# 重载运算符

只能重载部分已有的

不能创建新的

需要返回一个新对象

## 1 加法中辍运算符

In [13]:
import math
from array import array
import reprlib
import itertools

class Vector:

    typecode = 'd'

    def __init__(self, componets):
        self._componets = array(self.typecode, componets)

    def __iter__(self):
        return iter(self._componets)

    def __len__(self):
        return len(self._componets)

    def __getitem__(self, index):
        return self._componets[index]

    def __repr__(self):
        componets = reprlib.repr(self._componets)
        componets = componets[componets.find('['):-1]
        return "myVector({})".format(componets)

    def __str__(self):
        return str(tuple(self))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.sqrt(sum((x*x for x in self._componets)))

    """
        # 重载加法运算符
    def __add__(self, other):
        pairs = itertools.zip_longest(self, other, fillvalue=0.0)
        return Vector(a + b for a,b in pairs) # 运算符返回一个新对象
    """
    # 为了让解释器进行运算时，尽量使用到左右两个变量的运算符，而不是返回一些莫名其妙的错误，
    # 因此我们需要返回一个特殊的保持来是解释器继续工作。
    def __add__(self, other):
        try:
            pairs = itertools.zip_longest(self, other, fillvalue=0.0)
            return Vector(a + b for a,b in pairs) # 运算符返回一个新对象
        except TypeError:
            return NotImplemented
    # 于类型不兼容而导致运算符特殊方法无法返回有效的结果，那么应该返
    # 回 NotImplemented，而不是抛出 TypeError。返回
    # NotImplemented 时，另一个操作数所属的类型还有机会执行运算，
    # 即 Python 会尝试调用反向方法。

    # 重载右向运算符
    def __radd__(self, other):
        return self + other # 通常就是这么简单，直接委托给__add__方法

    def __bool__(self):
        return bool(abs(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)])+
                bytes(self._componets))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        octets = octets[1:]
        mev = memoryview(octets).cast(typecode)
        return cls(mev)

In [14]:
v1 = Vector([1.0,2.0,3.0])
v2 = Vector([5.0,6.0])

v1 + v2

myVector([6.0, 8.0, 3.0])

In [9]:
v1 + [5.0,6.0]

myVector([6.0, 8.0, 3.0])

In [6]:
# 左操作符。此时用的tuple的加法方法, 需要实现右操作符来规避这个问题。
# 这里涉及到python为中辍运算符的特殊分派机制（查找方法的顺序）
(5.0,6.0) + v1
# Error
"""
---------------------------------------------------------------------------

TypeError                                 Traceback (most recent call last)

<ipython-input-6-298acbd404ce> in <module>
----> 1 (5.0,6.0) + v1
      2

TypeError: can only concatenate tuple (not "Vector") to tuple
"""

TypeError: can only concatenate tuple (not "Vector") to tuple

In [10]:
# 实现右向运算符 __radd__就可以处理这样的问题：
(5.0,6.0) + v1

myVector([6.0, 8.0, 3.0])

In [11]:
v1 + 1

"""Error

---------------------------------------------------------------------------

TypeError                                 Traceback (most recent call last)

<ipython-input-11-4c3a4f2e82c5> in <module>
----> 1 v1 + 1

<ipython-input-7-585d1167aaf1> in __add__(self, other)
     36     # 重载加法运算符
     37     def __add__(self, other):
---> 38         pairs = itertools.zip_longest(self, other, fillvalue=0.0)
     39         return Vector(a + b for a,b in pairs) # 运算符返回一个新对象
     40

TypeError: 'int' object is not iterable
"""

TypeError: 'int' object is not iterable

In [15]:
# 修改__add__的函数，使其当TypeError时，返回NotImplemented，来让解释器进而尝试使用两个参数的
# 右向运算符来获得结果
v1 + 1

# 此时返回的错误会比较明确： unsupported operand type(s) for +: 'Vector' and 'int'

# 为什么不进行类型检查在相加，这是因为这样会违背鸭子类型原则。

TypeError: unsupported operand type(s) for +: 'Vector' and 'int'

## 2 乘法中辍运算符

In [20]:
import math
from array import array
import reprlib
import itertools
import numbers

class Vector:

    typecode = 'd'

    def __init__(self, componets):
        self._componets = array(self.typecode, componets)

    def __iter__(self):
        return iter(self._componets)

    def __len__(self):
        return len(self._componets)

    def __getitem__(self, index):
        return self._componets[index]

    def __repr__(self):
        componets = reprlib.repr(self._componets)
        componets = componets[componets.find('['):-1]
        return "myVector({})".format(componets)

    def __str__(self):
        return str(tuple(self))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.sqrt(sum((x*x for x in self._componets)))

    def __add__(self, other):
        try:
            pairs = itertools.zip_longest(self, other, fillvalue=0.0)
            return Vector(a + b for a,b in pairs) # 运算符返回一个新对象
        except TypeError:
            return NotImplemented

    def __radd__(self, other):
        return self + other # 通常就是这么简单，直接委托给__add__方法
# ----------------乘法中辍运算符-----------------
    def __mul__(self, other):
        if isinstance(other, numbers.Real): # 使用白鹅类型，利用抽线基类来进行类型判断
            return Vector( n * other for n in self)
        else:
            return NotImplemented

    def __rmul__(self, other):
        return self * other
# ++++++++++++++乘法中辍运算符+++++++++++++++++

# ----------------点积运算符-------------------
    def __matmul__(self, other):
        try:
            return sum( a * b for  a, b in zip(self, other))
        except TypeError:
            return NotImplemented

    def __rmatmul__(self, other):
        return self @ other
# ++++++++++++++++点积运算符++++++++++++++++++++

    def __bool__(self):
        return bool(abs(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)])+
                bytes(self._componets))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        octets = octets[1:]
        mev = memoryview(octets).cast(typecode)
        return cls(mev)


In [21]:
v3 = Vector([1.0,2.0,3.0])
v3 * 3.0

myVector([3.0, 6.0, 9.0])

In [22]:
v3 * 'A'
# Error: TypeError: can't multiply sequence by non-int of type 'Vector'

TypeError: can't multiply sequence by non-int of type 'Vector'

In [24]:
v4 = Vector([4.0,5.0, 6.0])
v3 @ v4

32.0

In [25]:
v3 @ 3
# TypeError: unsupported operand type(s) for @: 'Vector' and 'int'

TypeError: unsupported operand type(s) for @: 'Vector' and 'int'

## 3 比较运算符
- 正向和反向调用使用的是同一系列方法。这方面的规则如表 13-2
所示。例如，对 == 来说，正向和反向调用都是 __eq__ 方法，只
是把参数对调了；而正向的 __gt__ 方法调用的是反向的 __lt__
方法，并把参数对调。
- 对 == 和 != 来说，如果反向调用失败，Python 会比较对象的 ID，
而不抛出 TypeError。


In [27]:
import math
from array import array
import reprlib
import itertools
import numbers

class NewVector:

    typecode = 'd'

    def __init__(self, componets):
        self._componets = array(self.typecode, componets)

    def __iter__(self):
        return iter(self._componets)

    def __len__(self):
        return len(self._componets)

    def __getitem__(self, index):
        return self._componets[index]

    def __repr__(self):
        componets = reprlib.repr(self._componets)
        componets = componets[componets.find('['):-1]
        return "myVector({})".format(componets)

    def __str__(self):
        return str(tuple(self))
# ----------------比较运算符-----------------
    # 旧的方法会导致不同类型的数也会和该类实例相等，为了避免此类问题发生，我们需要必要的类型检查：
    def __eq__(self, other):
        #return tuple(self) == tuple(other)
        if isinstance(other, NewVector):
            return (len(self) == len(other)
                    and all(a == b for a , b in zip(self, other)))
        else:
            return NotImplemented

    # 不等其实可以不用实现，python会按照下面的方法自动实现！
    def __ne__(self, other):
        re = self == other
        if re is NotImplemented:
            return NotImplemented
        else :
            return not re
# +++++++++++++++++比较运算符++++++++++++++++
    def __abs__(self):
        return math.sqrt(sum((x*x for x in self._componets)))

    def __add__(self, other):
        try:
            pairs = itertools.zip_longest(self, other, fillvalue=0.0)
            return Vector(a + b for a,b in pairs) # 运算符返回一个新对象
        except TypeError:
            return NotImplemented

    def __radd__(self, other):
        return self + other # 通常就是这么简单，直接委托给__add__方法
# ----------------乘法中辍运算符-----------------
    def __mul__(self, other):
        if isinstance(other, numbers.Real): # 使用白鹅类型，利用抽线基类来进行类型判断
            return Vector( n * other for n in self)
        else:
            return NotImplemented

    def __rmul__(self, other):
        return self * other
# ++++++++++++++乘法中辍运算符+++++++++++++++++

# ----------------点积运算符-------------------
    def __matmul__(self, other):
        try:
            return sum( a * b for  a, b in zip(self, other))
        except TypeError:
            return NotImplemented

    def __rmatmul__(self, other):
        return self @ other
# ++++++++++++++++点积运算符++++++++++++++++++++

    def __bool__(self):
        return bool(abs(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)])+
                bytes(self._componets))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        octets = octets[1:]
        mev = memoryview(octets).cast(typecode)
        return cls(mev)


In [31]:
v5 = NewVector(range(1,5))
v6 = (1,2,3,4)
v5 == v6
# 这里由于双方的eq方法都可能返回NotImplemented，因此解释器可能回去比较二者的ID

False

In [32]:
v7 = NewVector(v6)
v7 == v5


True

In [33]:
# 这里NewVector的eq返回的是NotImplemented,因此解释器就去找Vector的eq方法了，而Vector的
# eq方法还是老方法，所以结果为True
v8 = Vector(v6)
v5 == v8

True

## 4 增量赋值运算符
如果一个类没有实现表 13-1 列出的就地运算符，增量赋值运算符只是
语法糖：a += b 的作用与 a = a + b 完全一样。对不可变类型来
说，这是预期的行为，而且，如果定义了 \_\_add__ 方法的话，不用编
写额外的代码，+= 就能使用。
然而，如果实现了就地运算符方法，例如 \_\_iadd__，计算 a += b 的
结果时会调用就地运算符方法。这种运算符的名称表明，它们会就地修
改左操作数，而不会创建新对象作为结果。

In [45]:
from chapter11_Tombola import addableBingoCage
vowels = 'AEIOU'
a = addableBingoCage.AddableBingoCage(vowels)
b = addableBingoCage.AddableBingoCage('abcd')

c = a + b
c.inspect()

('A', 'E', 'I', 'O', 'U', 'a', 'b', 'c', 'd')

In [46]:
a += c
a.inspect()

('A', 'A', 'E', 'E', 'I', 'I', 'O', 'O', 'U', 'U', 'a', 'b', 'c', 'd')

In [47]:
a.pick()

'c'

In [49]:
a += 'xyz'
a.inspect()

('A',
 'A',
 'E',
 'E',
 'I',
 'I',
 'O',
 'O',
 'U',
 'U',
 'a',
 'b',
 'd',
 'x',
 'x',
 'y',
 'y',
 'z',
 'z')

In [50]:
a += 1
# TypeError: right operand in += must be 'AddableBingoCage' or an iterable

TypeError: right operand in += must be 'AddableBingoCage' or an iterable